In [1]:
%pwd

'/home/larry/tensorflow_prac'

In [2]:
import numpy as np
import tensorflow as tf
import utils

In [3]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [4]:

batch_size = 128


In [5]:
# Step 1: Read in data
mnist_folder = 'data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


### What do  we get ?

In [6]:
type(train)

tuple

In [7]:
len(train)

2

In [8]:
ary1=train[0]

In [9]:
ary1.ndim

2

In [10]:
ary1.shape

(55000, 784)

In [11]:
ary2=train[1]
print(ary2.ndim)
print(ary2.shape)

2
(55000, 10)


In [28]:
type(test)

tuple

In [29]:
len(test)

2

In [30]:
test[0].shape

(10000, 784)

In [31]:
n_test = 10000

### using tf.data.Dataset.from_tensor_slices
### Creates a Dataset whose elements are slices of the given tensors.

In [12]:
train_data = tf.data.Dataset.from_tensor_slices(train)
test_data = tf.data.Dataset.from_tensor_slices(test)

In [13]:
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

### using tf.data.Iterator.from_structure
### Creates a new, uninitialized Iterator with the given structure.
### The returned iterator is not bound to a particular dataset, and it has no initializer. To initialize the iterator, run the operation returned by Iterator.make_initializer(dataset).

In [14]:
print(test_data.output_types)
print(test_data.output_shapes)

(tf.float32, tf.float64)
(TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


In [15]:
print(train_data.output_types)
print(train_data.output_shapes)

(tf.float32, tf.float64)
(TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


In [16]:
iterator = tf.data.Iterator.from_structure(train_data.output_types,
                                           train_data.output_shapes)

In [17]:
train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
img, label = iterator.get_next()

###   Let's build the graph:           logits = img * w +b 

In [19]:
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.random_normal_initializer(0, 0.01) )
logits = tf.matmul(img, w)+b

In [21]:
# define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch

In [23]:
# define training op
# using gradient descent with learning rate of 0.01 to minimize loss
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [24]:
# calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [34]:
n_epochs = 30
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # train the model n_epochs times
    for i in range(n_epochs):
        sess.run(train_init)
        total_loss=0
        n_batches=0
        try:
            while True:
                _,l=sess.run([optimizer, loss])
                total_loss  += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i,total_loss/n_batches))
    
    # test the model
    sess.run(test_init)# drawing samples from test_data
    total_correct_preds =  0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
        
    print('Accuracy {0}'.format(total_correct_preds/n_test))

Average loss epoch 0: 0.3636743127086828
Average loss epoch 1: 0.29206947119430055
Average loss epoch 2: 0.2822070433130098
Average loss epoch 3: 0.27669608979377636
Average loss epoch 4: 0.27294436691458834
Average loss epoch 5: 0.2701197351480639
Average loss epoch 6: 0.26786415226930793
Average loss epoch 7: 0.26598871067859403
Average loss epoch 8: 0.26438332399310066
Average loss epoch 9: 0.26297981286811273
Average loss epoch 10: 0.26173400891035103
Average loss epoch 11: 0.2606160241504048
Average loss epoch 12: 0.2596045637026776
Average loss epoch 13: 0.2586835312981938
Average loss epoch 14: 0.2578399938379609
Average loss epoch 15: 0.2570632369712342
Average loss epoch 16: 0.25634461372744205
Average loss epoch 17: 0.25567725207223446
Average loss epoch 18: 0.2550554196501887
Average loss epoch 19: 0.2544741517929144
Average loss epoch 20: 0.253929081108681
Average loss epoch 21: 0.25341647338382034
Average loss epoch 22: 0.25293322763470716
Average loss epoch 23: 0.25247695